### Hyperparameters

In [208]:
import torch

batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200

has_cuda = torch.cuda.is_available()
if not has_cuda:
    raise Exception("CUDA is not available")

device = 'cuda'

In [209]:
import requests

def download_tinyshakespeare():
	url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
	response = requests.get(url)

	if response.status_code == 200:
		with open("/data/tinyshakespeare.txt", "wb") as file:
			file.write(response.content)
			print("File downloaded successfully.")
	else:
		print("Failed to download the file.")

In [25]:
with open ('/data/tinyshakespeare.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [26]:
print("Characters: ", len(text))

Characters:  1115394


In [27]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [35]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("Vocabulary size:", vocab_size)
print("Vocabulary: ", ''.join(chars))

Vocabulary size: 65
Vocabulary:  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


### Encoder/decoder

In [18]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s] # encoder: map string to char ids
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: map integer ids to a string

In [120]:
print(encode("hi there"))
print(decode([46, 47, 1, 58, 46, 43, 56, 43]))

print(decode([0])) # Newline?
print(decode([1])) # Space?

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


 


In [20]:
# encoding the dataset and transforming it into a torch tensor
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [21]:
# Splitting the dataset into training and validation sets
n = int(0.9 * len(data)) # 90% training, 10% validation
train_data, validation_data = data[:n], data[n:]

In [22]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [24]:
# Visualising how chunking translates into learnable samples
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target=y[t]

  print(f"context: {context}, target: {target}")

context: tensor([18]), target: 47
context: tensor([18, 47]), target: 56
context: tensor([18, 47, 56]), target: 57
context: tensor([18, 47, 56, 57]), target: 58
context: tensor([18, 47, 56, 57, 58]), target: 1
context: tensor([18, 47, 56, 57, 58,  1]), target: 15
context: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47
context: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58


In [ ]:
# Batching/chunking the dataset for feeding it to the GPU
torch.manual_seed(42)

batch_size = 4 # independent sequences to be processed in parallel
block_size = 8 # maximum context length within each sequence

def get_batch(split):
  # batch of data of inputs x and targets y
  data = train_data if split == 'train' else validation_data
  # array(batch_size) of random offsets within data
  ix = torch.randint(len(data) - block_size, (batch_size,))

  # Context and target sequences
  # ? Why a single sequence is not used?
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)

print('\ntargets:')
print(yb.shape)
print(yb)

print('---')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]

    print(f"context: {context}, target: {target}")


inputs:
torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])

targets:
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
---
context: tensor([57]), target: 1
context: tensor([57,  1]), target: 46
context: tensor([57,  1, 46]), target: 47
context: tensor([57,  1, 46, 47]), target: 57
context: tensor([57,  1, 46, 47, 57]), target: 1
context: tensor([57,  1, 46, 47, 57,  1]), target: 50
context: tensor([57,  1, 46, 47, 57,  1, 50]), target: 53
context: tensor([57,  1, 46, 47, 57,  1, 50, 53]), target: 60
context: tensor([1]), target: 58
context: tensor([ 1, 58]), target: 46
context: tensor([ 1, 58, 46]), target: 43
context: tensor([ 1, 58, 46, 43]), target: 56
context: tensor([ 1, 58, 46, 43, 56]), target: 43
context: te

In [110]:
# Decode sample batches
print([decode(x) for x in xb.tolist()])

['s his lo', ' there c', 'ENCE:\nTo', 'st, with']


In [111]:
print(xb)
print('---')
print(yb)

tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
---
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])


In [81]:
# Bigram language model

import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()

    # Video says: each token directly reads off the logits for the next token
    # from the lookup table.
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    # idx and targets are (B, T) tensors of integers, Batch, Time
    # Token embedding table will return shape of (Batch, Time, Channel)
    # Batch - number of sequences
    # Time - number of tokens in each sequence
    # Channel - Embeddings vector for a token?
    logits = self.token_embedding_table(idx)

    if targets is None:
      return logits, None

    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)

    # Cross entropy expects input to be in a 2D tensor of shape (Batch*Time, Channel)
    loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is a (Batch, Time) tensor of integers, representing current context
    for _ in range(max_new_tokens):
      # compute the predictions
      logits, loss = self(idx) # (B, T, C)

      # -1 makes very little sense for a bigram model,
      # as we're essentially throwing away everything except the very last token in a batch
      # to make our prediction.
      # This is done in such a way only to allow for easier transition to an N-gram model later.
      logits = logits[:, -1, :] # (B, C)

      # Probabilities from logits
      # Softmax ~ [1, 2, 3, 4]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

      # append to the currently running context
      idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)

    return idx

In [82]:
# Init a model, sample forward pass
m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)

print(loss)
print(out.shape)

tensor(4.8865, grad_fn=<NllLossBackward0>)
torch.Size([32, 65])


In [230]:
# Sample generate
idx = torch.zeros((1, 1), dtype=torch.long)
logits = m.generate(idx, max_new_tokens=100)[0]

print(
  decode(
    logits.tolist()
  )
)


HOrty s ntecoan:
LAMing
Angais s anonpef, nerend awead.
CEThir my,
Many is h f? ie mos,

Is mbount b


In [210]:
# Optimizer takes the gradients and updates the model parameters
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-2)

In [229]:
# Sample training loop
batch_size = 32
train_steps = 10_000

for steps in range(train_steps):
  # sample a batch of data
  xb, yb = get_batch('train')
  xb.to(device)
  yb.to(device)

  # eval
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.5263893604278564


### Self-attention trick

In [231]:
torch.manual_seed(42)

B, T, C = 4, 8, 2 # Batch, Time, Channel
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [243]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)

xbow2 = wei @ x # (T, T) @ (B, T, C) ---> (B, T, C)

torch.allclose(xbow, xbow2)

True

In [233]:
# We want x[b, t] = mean_{i < t} x[b, i]
# i.e. for each sequence in the batch, we want to compute the mean of all the previous tokens for each token.

xbow = torch.zeros((B, T, C))

for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1] # (t, C)
    xbow[b, t] = torch.mean(xprev, 0)

In [241]:
torch.manual_seed(42)

a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print('a=', a)
print('b=', b)
print('c=', c)

a= tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c= tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [244]:
# version 3, using Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)

xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [253]:
# version 3, Self-attention
torch.manual_seed(42)

B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# Single head self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)


# Communication: all q and k pairs to make dot products
wei = q @ k.transpose(-2, -1) # (B, T, 16 @ (B, 16, T) -> (B, T, T)
# B -> (T, T) - where T^2 is affinity matrix on how q and k pairs related to each other

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)

v = value(x)
out = wei @ v

wei[0]

tensor([[0.0839, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0606, 0.2418, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1684, 0.0240, 0.2433, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1060, 0.2614, 0.1076, 0.3452, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1187, 0.0218, 0.0152, 0.0890, 0.0695, 0.0000, 0.0000, 0.0000],
        [0.0654, 0.3663, 0.0801, 0.3100, 0.6478, 0.0812, 0.0000, 0.0000],
        [0.3644, 0.0150, 0.5266, 0.0205, 0.1464, 0.0097, 0.0605, 0.0000],
        [0.0326, 0.0698, 0.0271, 0.2353, 0.1363, 0.9091, 0.9395, 1.0000]],
       grad_fn=<SelectBackward0>)